# SETUP VIZDOOM FOR PYTHON

In [18]:
from typing import Tuple

from gym.core import ActType, ObsType
!pip install vizdoom

  Obtaining dependency information for vizdoom from https://files.pythonhosted.org/packages/89/8c/797786b5523deba5c9b21adead674a3f47dd5f15ec65d4379de5ffd81566/vizdoom-1.2.3-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for gymnasium>=0.28.0 from https://files.pythonhosted.org/packages/a8/4d/3cbfd81ed84db450dbe73a89afcd8bc405273918415649ac6683356afe92/gymnasium-0.29.1-py3-none-any.whl.metadata
  Obtaining dependency information for pygame>=2.1.3 from https://files.pythonhosted.org/packages/82/61/93ae7afbd931a70510cfdf0a7bb0007540020b8d80bc1d8762ebdc46479b/pygame-2.5.2-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/15.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.5 MB 1.3 MB/s eta 0:00:13
   ---------------------------------------- 0.1/15.5 MB 2.1 MB/s eta 0:00:08
    --------------------------------------- 0.3/15.5 MB 3.1 MB/s eta 0:00:05
   - -------------------------------------- 0.6/15.5 MB 3.6 MB/s e

In [20]:
!cd github & git clone https://github.com/mwydmuch/ViZDoom


Cloning into 'ViZDoom'...
Updating files:  48% (997/2063)
Updating files:  49% (1011/2063)
Updating files:  50% (1032/2063)
Updating files:  51% (1053/2063)
Updating files:  52% (1073/2063)
Updating files:  53% (1094/2063)
Updating files:  54% (1115/2063)
Updating files:  55% (1135/2063)
Updating files:  56% (1156/2063)
Updating files:  57% (1176/2063)
Updating files:  58% (1197/2063)
Updating files:  59% (1218/2063)
Updating files:  60% (1238/2063)
Updating files:  61% (1259/2063)
Updating files:  62% (1280/2063)
Updating files:  63% (1300/2063)
Updating files:  64% (1321/2063)
Updating files:  65% (1341/2063)
Updating files:  66% (1362/2063)
Updating files:  67% (1383/2063)
Updating files:  68% (1403/2063)
Updating files:  69% (1424/2063)
Updating files:  70% (1445/2063)
Updating files:  71% (1465/2063)
Updating files:  72% (1486/2063)
Updating files:  73% (1506/2063)
Updating files:  74% (1527/2063)
Updating files:  75% (1548/2063)
Updating files:  76% (1568/2063)
Updating files:  7

In [1]:
# Import vizdoom for game env
from vizdoom import * 
# Import random for action sampling
import random
# Import time for sleeping
import time 
# Import numpy for identity matrix
import numpy as np

In [2]:
game = DoomGame()

game.load_config('github/VizDoom/scenarios/basic.cfg')
game.init()

In [3]:
actions = np.identity(3, dtype=np.uint8)


In [4]:
state = game.get_state()

In [5]:
state.game_variables

array([50.])

In [6]:
# Loop Through Episodes

episodes = 10
for episode in range(episodes):
    game.new_episode()
    while not game.is_episode_finished():
        state = game.get_state()
        img = state.screen_buffer
        info = state.game_variables
        reward = game.make_action(random.choice(actions),4)
        print('reward:', reward)
        time.sleep(0.02)
    print('Result:', game.get_total_reward())
    time.sleep(2)
    

reward: -4.0
reward: 99.0
Result: 95.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -9.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -9.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -9.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -9.0
reward: -4.0
reward: -4.0
reward: -9.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -9.0
reward: -4.0
reward: -4.0
reward: -9.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -9.0
reward: -4.0
reward: -4.0
reward: -9.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -9.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -9.0
reward: -4.0
reward: -4.0
reward: -9.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -9.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -9.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: 98.0
Result: -232.0
reward: -4.0
reward: 99.0
Result: 95.0
reward: -4.0
reward: -4.0
reward: -4.0
reward: -4

ViZDoomUnexpectedExitException: Controlled ViZDoom instance exited unexpectedly.

In [ ]:
game.close()

# 2. Converting it to a Gym Environment

In [39]:
!pip install gym

In [7]:
from gym import Env

from gym.spaces import Discrete, Box

import cv2

In [8]:
# Create Vizdoom OpenAI Gym Environment
class VizDoomGym(Env): 
    # Function that is called when we start the env
    def __init__(self, render=False): 
        # Inherit from Env
        super().__init__()
        # Setup the game 
        self.game = DoomGame()
        self.game.load_config('github/VizDoom/scenarios/defend_the_center.cfg')
        
        # Render frame logic
        if render == False: 
            self.game.set_window_visible(False)
        else:
            self.game.set_window_visible(True)
        
        # Start the game 
        self.game.init()
        
        # Create the action space and observation space
        self.observation_space = Box(low=0, high=255, shape=(100,160,1), dtype=np.uint8) 
        self.action_space = Discrete(3)
        
    # This is how we take a step in the environment
    def step(self, action):
        # Specify action and take step 
        actions = np.identity(3)
        reward = self.game.make_action(actions[action], 4) 
        
        # Get all the other stuff we need to retun 
        if self.game.get_state(): 
            state = self.game.get_state().screen_buffer
            state = self.grayscale(state)
            ammo = self.game.get_state().game_variables[0]
            info = ammo
        else: 
            state = np.zeros(self.observation_space.shape)
            info = 0 
        
        info = {"info":info}
        done = self.game.is_episode_finished()
        
        return state, reward, done, info 
    
    # Define how to render the game or environment 
    def render(): 
        pass
    
    # What happens when we start a new game 
    def reset(self): 
        self.game.new_episode()
        state = self.game.get_state().screen_buffer
        return self.grayscale(state)
    
    # Grayscale the game frame and resize it 
    def grayscale(self, observation):
        gray = cv2.cvtColor(np.moveaxis(observation, 0, -1), cv2.COLOR_BGR2GRAY)
        resize = cv2.resize(gray, (160,100), interpolation=cv2.INTER_CUBIC)
        state = np.reshape(resize, (100,160,1))
        return state
    
    # Call to close down the game
    def close(self): 
        self.game.close()

In [9]:
env = VizDoomGym(render=True)

In [ ]:
state = env.reset()

In [11]:
env.step(0)

(array([[[10],
         [ 4],
         [ 7],
         ...,
         [10],
         [ 6],
         [10]],
 
        [[ 6],
         [ 9],
         [ 8],
         ...,
         [ 5],
         [ 7],
         [10]],
 
        [[ 8],
         [ 9],
         [ 8],
         ...,
         [10],
         [ 6],
         [ 6]],
 
        ...,
 
        [[75],
         [63],
         [62],
         ...,
         [44],
         [71],
         [60]],
 
        [[15],
         [48],
         [47],
         ...,
         [49],
         [69],
         [47]],
 
        [[22],
         [14],
         [26],
         ...,
         [57],
         [37],
         [39]]], dtype=uint8),
 0.0,
 False,
 {'info': 26.0})

In [12]:
env.close()